<a href="https://colab.research.google.com/github/jetsonmom/2025_0623_chungnam_automobile/blob/main/cuda%26ufunc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
import numba
from numba import cuda

if cuda.is_available():
  print("🎉 성공: Numba가 GPU를 성공적으로 인식했습니다!")
  print("이제 ufunc 등 모든 CUDA 코드를 실행할 수 있습니다.")
  print("-" * 50)
  # Numba가 인식한 GPU의 상세 정보를 출력합니다.
  cuda.detect()
else:
  # 거의 발생하지 않겠지만, 만약을 위한 오류 메시지
  print("오류: 아직 Numba가 GPU를 인식하지 못합니다. 런타임을 다시 초기화해보세요.")

In [ ]:
from numba import cuda

if cuda.is_available():
  print("성공! Numba가 GPU를 인식했습니다.")
  print("GPU 장치 정보:")
  cuda.detect()
else:
  print("오류: Numba가 GPU를 인식하지 못했습니다.")

In [ ]:
import numpy as np
from numba import vectorize

# ufunc 정의: 각 입력값(x)에 2를 곱하고 1을 더하는 함수
# target='cuda' 옵션으로 이 함수를 GPU에서 실행하도록 지정합니다.
@vectorize(['float32(float32)'], target='cuda')
def add_two_and_one(x):
  return x * 2 + 1

# 0부터 9까지의 숫자로 배열 생성
data = np.arange(10, dtype=np.float32)

print("원본 데이터:", data)

# GPU에서 ufunc 실행
result = add_two_and_one(data)

print("GPU 연산 결과:", result)

In [ ]:
## See: https://github.com/googlecolab/colabtools/issues/5081#issuecomment-2629611179
!uv pip install -q --system numba-cuda==0.4.0
from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1
config.CUDA_LOW_OCCUPANCY_WARNINGS = 0

In [ ]:
import numba
import numpy as np
from numba import vectorize, jit, cuda
print("Numpy version: ", np.__version__)
print("numba version: ", numba.__version__)
print("Cuda avilable") if cuda.detect() else print("Cuda not avilable")

# Day 5: CUDA 파이썬을 이용한 자율주행 데이터 가속

**과정 목표:** 지난 4일간 배운 자율주행 데이터 처리 알고리즘들을 GPU를 이용해 가속하는 방법을 배웁니다. Numba 라이브러리를 사용하여 Python으로 직접 CUDA 코드를 작성하고, 병렬 처리의 핵심 원리와 최적화 기법을 실습을 통해 익힙니다.

**과정 목표:** 지난 4일간 배운 자율주행 데이터 처리 알고리즘들을 GPU를 이용해 가속하는 방법을 배웁니다. Numba 라이브러리를 사용하여 Python으로 직접 CUDA 코드를 작성하고, 병렬 처리의 핵심 원리와 최적화 기법을 실습을 통해 익힙니다.
---
## Lab 1: Numba Ufunc, 프로파일링, 그리고 정밀도

**실습 목표:**
1. 간단한 이미지 처리 작업을 위한 GPU 범용 함수(ufunc)를 작성합니다.
2. 데이터 크기에 따른 CPU와 GPU의 성능을 `%%timeit`으로 비교하여 메모리 전송 오버헤드를 확인합니다.
3. `float32`와 `float64`의 정밀도 차이가 계산 결과에 미치는 영향을 직접 확인합니다.
### 준비: 라이브러리 임포트 및 데이터 생성

In [ ]:
import numpy as np
from numba import vectorize, jit
import time

# 자율주행 카메라 이미지를 모방한 가상 데이터 생성
def create_image(size_mb):
    # 1 pixel = 1 byte (uint8)
    num_pixels = size_mb * 1024 * 1024
    # 이미지의 가로:세로 비율을 16:9로 가정
    height = int(np.sqrt(num_pixels * 9 / 16))
    width = int(height * 16 / 9)
    print(f"생성된 이미지 크기: {width}x{height} ({width*height/1024/1024:.2f} MB)")
    return np.random.randint(0, 256, size=(height, width), dtype=np.uint8)

small_image = create_image(1)    # 약 1MB 크기 이미지
large_image = create_image(100)  # 약 100MB 크기 이미지

In [ ]:
# TODO: 아래 함수를 Numba의 @vectorize 데코레이터를 사용하여 GPU ufunc으로 변환하세요.
# target을 'cuda'로 설정하는 것을 잊지 마세요.
# 타입 시그니처: uint8를 입력받아 uint8를 반환 -> ['uint8(uint8, uint8)']

from numba import vectorize, uint8, cuda

@vectorize([uint8(uint8, uint8)], target='cuda')
def gpu_threshold(pixel, threshold):
    return 255 if pixel > threshold else 0

# CPU 버전 (NumPy)
def cpu_threshold_numpy(image, threshold):
    return np.where(image > threshold, 255, 0).astype(np.uint8)

In [ ]:
print("--- 작은 이미지 (1MB) 성능 비교 ---")
print("CPU (NumPy):")
%timeit cpu_threshold_numpy(small_image, 128)

print("\nGPU (Numba ufunc):")
# 첫 실행은 컴파일 시간 포함
_ = gpu_threshold(small_image, 128)
%timeit gpu_threshold(small_image, 128)



In [ ]:
print("--- 큰 이미지 (100MB) 성능 비교 ---")
print("CPU (NumPy):")
%timeit cpu_threshold_numpy(large_image, 128)

print("\nGPU (Numba ufunc):")
_ = gpu_threshold(large_image, 128)
%timeit gpu_threshold(large_image, 128)

In [ ]:
import numpy as np
from numba import vectorize

# Corrected version to demonstrate precision loss
@vectorize(['float32(float32)'], target='cuda')
def precision_test_f32_corrected(x):
    # Force the literal '1.0' to be a 32-bit float
    one_f32 = np.float32(1.0)
    return (one_f32 + x) - one_f32

@vectorize(['float64(float64)'], target='cuda')
def precision_test_f64(x):
    # For float64, using a Python literal is fine as it's already 64-bit
    return (1.0 + x) - 1.0

# Using a value that WILL be lost in float32 but not float64
# float32 machine epsilon is ~1.19e-7. 1e-8 is smaller than that.
val = 1e-8
x_f32 = np.array([val], dtype=np.float32)
x_f64 = np.array([val], dtype=np.float64)

# Run the corrected f32 version and the f64 version
result_f32 = precision_test_f32_corrected(x_f32)
result_f64 = precision_test_f64(x_f64)

print(f"입력 값: {val}")
print(f"Corrected Float32 결과: {result_f32[0]}")
print(f"Float64 결과: {result_f64[0]}")